<a href="https://colab.research.google.com/github/Mojtaba-Choopani/huggingface-llm-course-fa-notebooks/blob/main/chapter3-FINE-TUNING-PERETRAINED_MODEL/section1-Processing-the-data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div dir="rtl">

<b style="font-size: 24px;"> پردازش داده‌ها </b>

</div>

# Processing the data (PyTorch)

<div dir="rtl"> <p> نحوه آموزش یک Sequence Classifier روی یک batch </p> </div>

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.optim import AdamW


# Same as before
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

# This is new
batch["labels"] = torch.tensor([1, 1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

<div dir="rtl">
  <p>
    در این بخش از مجموعه داده <strong>MRPC</strong> استفاده می‌کنیم که شامل ۵۸۰۱ جفت جمله با برچسب هم‌معنی بودن یا نبودن است. این مجموعه کوچک است و برای آزمایش و آموزش مدل مناسب می‌باشد.
  </p>
</div>
---

<div dir="rtl">
  <p>
    هاب Hugging Face علاوه بر مدل‌ها، شامل مجموعه داده‌های متنوع در زبان‌های مختلف نیز هست. در این بخش، تمرکز بر روی مجموعه داده <strong>MRPC</strong> است که یکی از ۱۰ مجموعه داده معیار <strong>GLUE</strong> برای ارزیابی عملکرد مدل‌های یادگیری ماشین در ۱۰ وظیفه دسته‌بندی متن می‌باشد. با کتابخانه 🤗 <strong>Datasets</strong> می‌توان به سادگی این مجموعه داده را دانلود و ذخیره محلی کرد.
  </p>
</div>


In [43]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

<div dir="rtl"> <p> با اجرای دستور مربوطه، یک <strong>DatasetDict</strong> دریافت می‌کنیم که شامل سه بخش است: <em>training</em>، <em>validation</em> و <em>test</em>. هر بخش چند ستون دارد: <strong>sentence1</strong>، <strong>sentence2</strong>، <strong>label</strong> و <strong>idx</strong> و تعداد سطرها با تعداد جفت‌های جمله در هر بخش برابر است (مثلاً ۳,۶۶۸ جفت در <em>training</em>، ۴۰۸ در <em>validation</em> و ۱,۷۲۵ در <em>test</em>). </p> <p> این دستور مجموعه داده را دانلود و در حافظه محلی <code>~/.cache/huggingface/datasets</code> ذخیره می‌کند. می‌توان مسیر کش را با تنظیم متغیر محیطی <strong>HF_HOME</strong> تغییر داد. </p> <p> برای دسترسی به هر جفت جمله در <strong>raw_datasets</strong> می‌توان از اندیس‌گذاری مانند دیکشنری استفاده کرد. </p> </div>

In [44]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

<div dir="rtl">
  <p>
    می‌بینیم که برچسب‌ها (<strong>labels</strong>) قبلاً به صورت عددی هستند، بنابراین نیازی به پیش‌پردازش روی آن‌ها نداریم. برای فهمیدن این که هر عدد به کدام برچسب مربوط است، می‌توانیم ویژگی‌های <strong>raw_train_dataset</strong> را بررسی کنیم. این کار نوع هر ستون را به ما نشان می‌دهد.
  </p>
</div>


<div dir="rtl">
    <p>
    در پشت صحنه، ستون <strong>label</strong> از نوع <code>ClassLabel</code> است و نگاشت اعداد به نام برچسب‌ها در فولدر <strong>names</strong> ذخیره شده است.
    عدد <strong>0</strong> به <code>not_equivalent</code> و عدد <strong>1</strong> به <code>equivalent</code> متناظر است.
  </p>
</div>


In [45]:
raw_train_dataset.features

{'sentence1': Value('string'),
 'sentence2': Value('string'),
 'label': ClassLabel(names=['not_equivalent', 'equivalent']),
 'idx': Value('int32')}

<div dir="rtl">
    
  <b style="font-size: 18px;"> آماده‌سازی داده‌ها</b>
  
     برای آماده‌سازی داده‌ها برای مدل، متن‌ها باید با توکنایزر به اعداد تبدیل شوند. این کار شامل توکنیزه کردن همه جمله‌های اول و دوم هر جفت جمله است.
  </p>
</div>


In [46]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"][:])
tokenized_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"][:])

<div dir="rtl">
  <p>
    برای پیش‌بینی اینکه دو جمله هم معنی هستند یا نه، نمی‌توانیم فقط دو جمله را جداگانه به مدل بدهیم. باید جفت جملات را به صورت یک ورودی واحد پردازش کنیم و توکنایزر می‌تواند این کار را مطابق انتظار مدل BERT انجام دهد.
  </p>
</div>


In [47]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

<div dir="rtl">
  <p>
    اگر مقادیر داخل <code>input_ids</code> را دوباره به کلمات تبدیل کنیم (decode کنیم):
  </p>
</div>


In [48]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[CLS]',
 'this',
 'is',
 'the',
 'first',
 'sentence',
 '.',
 '[SEP]',
 'this',
 'is',
 'the',
 'second',
 'one',
 '.',
 '[SEP]']

<div dir="rtl">
  <p>
    <code>token_type_ids</code> مشخص می‌کند کدام بخش از ورودی مربوط به جمله اول و کدام مربوط به جمله دوم است.
  </p>
  <p>
    همه مدل‌ها این مقادیر را ندارند؛ فقط مدل‌هایی که در پیش‌آموزش آن‌ها استفاده شده‌اند، این اطلاعات را نیاز دارند (مثل BERT).
  </p>
  <p>
    BERT علاوه بر مدلسازی زبان ماسک‌شده، وظیفه پیش‌بینی جمله بعدی هم دارد تا رابطه بین جفت جمله‌ها را یاد بگیرد.
  </p>
  <p>
    برای استفاده معمولی، کافی است از همان checkpoint برای توکنیزر و مدل استفاده کنید تا همه چیز درست کار کند.
  </p>
</div>
---

<div dir="rtl">
  <p>
    حالا که می‌دانیم توکنایزر چگونه یک جفت جمله را پردازش می‌کند، می‌توانیم کل مجموعه داده را توکنیزه کنیم.
  </p>
  <p>
    برای این کار، لیست جملات اول و لیست جملات دوم را به توکنایزر می‌دهیم.
  </p>
  <p>
    این روش با گزینه‌های <code>padding</code> و <code>truncation</code> که در فصل ۲ دیدیم، سازگار است.
  </p>
  <p>
    به این ترتیب، داده‌ها آماده ورودی مدل می‌شوند و پیش‌پردازش مجموعه آموزش انجام می‌شود.
  </p>
</div>


In [49]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"][:],
    raw_datasets["train"]["sentence2"][:],
    padding=True,
    truncation=True,
)

<div dir="rtl">
  <p>
    توکنیزه کردن همه داده‌ها همزمان حافظه زیادی نیاز دارد و خروجی‌اش یک دیکشنری از لیست‌هاست، نه یک <code>Dataset</code> قابل استفاده مستقیم.
  </p>
  <p>
    برای حفظ ساختار <code>Dataset</code> و استفاده بهینه از حافظه، از <code>Dataset.map()</code> استفاده می‌کنیم.
  </p>
  <p>
    این متد روی هر نمونه از مجموعه داده تابعی اعمال می‌کند، پس می‌توانیم یک تابع تعریف کنیم که توکنیزیشن و پیش‌پردازش دلخواه را انجام دهد.
  </p>
  <p>
    با استفاده از این روش، داده‌ها به شکل مناسبی برای مدل آماده می‌شوند و مصرف حافظه بهینه باقی می‌ماند.

   یک نمونه کد پایتون برای استفاده از Dataset.map() همراه با توکنایزر:

  </p>
</div>


In [50]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

<div dir="rtl">
  <p>
    تابعی که تعریف می‌کنیم، یک دیکشنری از داده‌ها می‌گیرد و خروجی آن دیکشنری‌ای با کلیدهای <code>input_ids</code>، <code>attention_mask</code> و <code>token_type_ids</code> است.
  </p>
  <p>
    این تابع می‌تواند روی چند نمونه به‌صورت هم‌زمان هم کار کند و به همین دلیل می‌توانیم در <code>Dataset.map()</code> از گزینه <code>batched=True</code> استفاده کنیم تا توکنیزه کردن سریع‌تر شود.
  </p>
  <p>
    توکنیزر استفاده شده از کتابخانه 🤗 Tokenizers است که با زبان Rust نوشته شده و برای پردازش همزمان تعداد زیادی ورودی بسیار سریع است.
  </p>
  <p>
    <strong>نکته:</strong> padding فعلاً در تابع اعمال نشده چون بهتر است هنگام ساختن batch انجام شود، نه برای کل مجموعه داده، تا مصرف حافظه و زمان کمتر شود.
  </p>
  <p>
    در نهایت، با استفاده از <code>batched=True</code> می‌توانیم تابع را روی همه داده‌ها اعمال کنیم و پیش‌پردازش سریع‌تری داشته باشیم.
  </p>
</div>


In [51]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

<div dir="rtl">
  <p>
    کتابخانه 🤗 Datasets هنگام اعمال پیش‌پردازش، داده‌های اصلی را تغییر نمی‌دهد، بلکه ستون‌های جدیدی به مجموعه داده اضافه می‌کند.
  </p>
  <p>
    هر کلید خروجی از تابع پیش‌پردازش (مثل <code>input_ids</code>، <code>attention_mask</code> و <code>token_type_ids</code>) به یک ستون جداگانه در Dataset تبدیل می‌شود.
  </p>
</div>
---

<div dir="rtl">
  <p>
    می‌توان از چندپردازشی (multiprocessing) هنگام استفاده از <code>map()</code> برای پیش‌پردازش استفاده کرد با پارامتر <code>num_proc</code>، اما اگر از توکنایزر سریع 🤗 استفاده می‌کنید، معمولاً لازم نیست چون خودش چند رشته‌ای کار می‌کند.
  </p>
  <p>
    تابع <code>tokenize_function</code> یک دیکشنری با کلیدهای <code>input_ids</code>، <code>attention_mask</code> و <code>token_type_ids</code> برمی‌گرداند که به تمام بخش‌های مجموعه داده اضافه می‌شوند.
  </p>
  <p>
    می‌توان مقادیر موجود در مجموعه داده را هم با همان کلیدها تغییر داد اگر تابع پیش‌پردازش مقدار جدید بدهد.
  </p>
  <p>
    آخرین مرحله <strong>dynamic padding</strong> است: هنگام ایجاد batch، همه نمونه‌ها به طول طولانی‌ترین عنصر در آن batch پر می‌شوند تا مدل بتواند به صورت هم‌زمان پردازش کند.
  </p>
</div>


<div dir="rtl">

  <b style="font-size: 18px;"> dynamic padding </b>
  <p>
    <strong>Collate function</strong>  نمونه‌ها را کنار هم در یک batch قرار می‌دهد. و برای ورودی‌هایی با طول متغیر، <strong>padding</strong> را به صورت دینامیک اعمال می‌کند تا آموزش سریع‌تر و بهینه‌تر انجام شود.
  </p>
</div>


In [52]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

<div dir="rtl">
  <p>
    برای آزمایش این ابزار جدید، چند نمونه از مجموعه آموزش خود انتخاب می‌کنیم تا با هم در یک <strong>batch</strong> قرار دهیم. در اینجا ستون‌های <code>idx</code>، <code>sentence1</code> و <code>sentence2</code> را حذف می‌کنیم، زیرا نیازی به آن‌ها نداریم و حاوی رشته هستند (و نمی‌توان تنسور از رشته‌ها ساخت). سپس طول هر نمونه در <strong>batch</strong> را بررسی می‌کنیم.
  </p>
</div>


In [53]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]

<div dir="rtl">
  <p>
    غیرمنتظره نیست که نمونه‌ها طول‌های متفاوتی دارند، از ۳۲ تا ۶۷. <strong>Dynamic padding</strong> یعنی نمونه‌های این <strong>batch</strong> باید همه تا طول ۶۷ (بیشترین طول در این batch) پر شوند. بدون dynamic padding، همه نمونه‌ها باید تا بیشترین طول کل مجموعه داده یا بیشترین طول قابل قبول مدل پر می‌شدند. حالا بیایید بررسی کنیم که <code>data_collator</code> ما به درستی <strong>batch</strong> را به‌صورت دینامیک پر می‌کند:
  </p>
</div>


In [54]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 67]),
 'token_type_ids': torch.Size([8, 67]),
 'attention_mask': torch.Size([8, 67]),
 'labels': torch.Size([8])}

<div dir="rtl">
  <p>
     حالا که از متن خام به <strong>batch</strong>هایی رسیدیم که مدل ما می‌تواند با آن‌ها کار کند، آماده‌ایم تا مدل را <strong>fine-tune</strong> کنیم!
  </p>
</div>
